In [13]:
import pandas as pd
import numpy as np
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [14]:
file_path_5 = Path("clean_suburbs_53.csv")
df_suberbs = pd.read_csv(file_path_5)

df_suberbs.head()

,year,month,suburb,medianSoldPrice,numberSold,highestSoldPrice,numberSaleListing,highestSaleListingPrice,lowestSaleListingPrice,auctionNumberAuctioned,auctionNumberSold,medianRentListingPrice
0,2011.0,8.0,Randwick,1422000.0,40.0,5450000.0,81.0,8000000.0,775000.0,32.0,20.0,950.0
1,2011.0,11.0,Randwick,1417000.0,49.0,3660000.0,97.0,6000000.0,700000.0,47.0,23.0,880.0
2,2012.0,2.0,Randwick,1520000.0,24.0,6200000.0,62.0,5850000.0,570000.0,10.0,5.0,975.0
3,2012.0,5.0,Randwick,1555000.0,44.0,3325000.0,77.0,3590000.0,570000.0,30.0,17.0,850.0
4,2012.0,8.0,Randwick,1385000.0,27.0,3050000.0,57.0,4250000.0,650000.0,19.0,11.0,825.0


Need to clean the data, and concatinate the data frames. 
From cash rate we need the second column. 
DPIE we need the Grand Total. 
AUD exhange rate will use USD which is the first column.
Will use the sydney CPI data from the CPI df, and its the first column of data (after the date)
Will use the percentage change in GDP, which is the third column in the dataframe


In [15]:
#df_suberbs_clean = df_suberbs.drop_duplicates(subset=['year'])
df_suberbs_clean = df_suberbs.drop_duplicates(subset=['suburb', 'year'])
df_suberbs_clean= df_suberbs_clean.iloc[:, :4]

df_suberbs_clean.head(20)

,year,month,suburb,medianSoldPrice
0,2011.0,8.0,Randwick,1422000.0
2,2012.0,2.0,Randwick,1520000.0
6,2013.0,2.0,Randwick,1525000.0
10,2014.0,2.0,Randwick,1590000.0
14,2015.0,2.0,Randwick,1820000.0
18,2016.0,2.0,Randwick,2420000.0
22,2017.0,2.0,Randwick,2270000.0
26,2018.0,2.0,Randwick,2260000.0
30,2019.0,2.0,Randwick,2325000.0
34,2020.0,2.0,Randwick,2801000.0


In [16]:
#clean cash rate data
file_path_1 = Path("Cash_rate.csv")
df_cash_rate = pd.read_csv(file_path_1, parse_dates=True)  #, index_col='Date'
df_cash_rate['Date'] = pd.to_datetime(df_cash_rate['Date']).dt.to_period('Y')
df_cash_rate.set_index('Date').resample('Y')
df_cash_rate = df_cash_rate.drop_duplicates(subset=['Date'])
df_cash_rate.rename(columns={'Cash_rate': 'Aus cash rate'})
df_cash_rate = df_cash_rate.set_index('Date')
df_cash_rate.head(20)

,Cash_rate
Date,
2011,4.75
2012,4.25
2013,3.00
2014,2.50
2015,2.50
2016,2.00
2017,1.50
2018,1.50
2019,1.50


In [80]:
#clean DPIE data
file_path_4 = Path("dpie_nsw_sydney_approvals_monthly.csv")
df_dpie = pd.read_csv(file_path_4, index_col='Year and Month', parse_dates=True, thousands=',') #,header=2, parse_dates=True, index_col='Year and Month'
df_dpie.tail()

,Detached,Unnamed: 2,Unnamed: 3
Year and Month,,,
2021-02-01,1333,NaN,NaN
2021-03-01,1539,NaN,NaN
2021-04-01,1620,NaN,NaN
2021-05-01,1861,NaN,NaN
2021-06-01,1367,NaN,NaN


In [81]:
df_dpie.index.names = ['Date']
df_dpie = df_dpie.reset_index()
df_dpie.head()

,Date,Detached,Unnamed: 2,Unnamed: 3
0,2011-01-01,526,NaN,NaN
1,2011-02-01,779,NaN,NaN
2,2011-03-01,827,NaN,NaN
3,2011-04-01,616,NaN,NaN
4,2011-05-01,731,NaN,NaN


In [82]:
df_dpie.set_index('Date')

,Detached,Unnamed: 2,Unnamed: 3
Date,,,
2011-01-01,526,NaN,NaN
2011-02-01,779,NaN,NaN
2011-03-01,827,NaN,NaN
2011-04-01,616,NaN,NaN
2011-05-01,731,NaN,NaN
...,...,...,...
2021-02-01,1333,NaN,NaN
2021-03-01,1539,NaN,NaN
2021-04-01,1620,NaN,NaN


In [83]:
df_dpie.tail()

,Date,Detached,Unnamed: 2,Unnamed: 3
121,2021-02-01,1333,NaN,NaN
122,2021-03-01,1539,NaN,NaN
123,2021-04-01,1620,NaN,NaN
124,2021-05-01,1861,NaN,NaN
125,2021-06-01,1367,NaN,NaN


In [84]:
df_dpie = df_dpie.iloc[:, :2]
df_dpie = df_dpie.rename(columns={'Detached': 'DPIE data'})
df_dpie['Date'] = pd.to_datetime(df_dpie['Date']).dt.to_period('Y')
df_dpie.set_index('Date').resample('Y')
df_dpie = df_dpie.drop_duplicates(subset=['Date'])
df_dpie = df_dpie.set_index('Date')

In [85]:
df_dpie.head()

,Detached
Date,
2011,526
2012,451
2013,579
2014,919
2015,877


In [86]:
#cleanse AUD exchange rate
file_path = Path("AUD_USD Historical Data.csv")
df_exchange_rates = pd.read_csv(file_path, index_col='Date', parse_dates=True) #
df_exchange_rates.head()

,Price
Date,
2011-01-01,1.0433
2012-01-01,1.0393
2013-01-01,0.8913
2014-01-01,0.8168
2015-01-01,0.7275


In [87]:
# df_exchange_rates = df_exchange_rates.iloc[:,[0]]
df_exchange_rates.head()

,Price
Date,
2011-01-01,1.0433
2012-01-01,1.0393
2013-01-01,0.8913
2014-01-01,0.8168
2015-01-01,0.7275


In [88]:
df_exchange_rates = df_exchange_rates.reset_index()
df_exchange_rates['Date'] =  pd.to_datetime(df_exchange_rates['Date']).dt.to_period('Y')
df_exchange_rates = df_exchange_rates.set_index('Date')
df_exchange_rates.columns = ['AUDUSD']
# df_exchange_rates = df_exchange_rates.drop_duplicates(subset=['Date'])
# df_exchange_rates = df_exchange_rates.set_index('Date')
df_exchange_rates.head()

,AUDUSD
Date,
2011,1.0433
2012,1.0393
2013,0.8913
2014,0.8168
2015,0.7275


In [89]:
#cleanse CPI data
file_path_2 = Path("CPI_y_10y_edit.csv")
df_CPI = pd.read_csv(file_path_2, index_col='Date', parse_dates=True)

In [90]:
df_CPI = df_CPI.reset_index()
df_CPI.head(12)

,Date,Sydney_CPI
0,2011-01-01,99.8
1,2012-01-01,102.3
2,2013-01-01,105.0
3,2014-01-01,106.8
4,2015-01-01,108.9
5,2016-01-01,110.9
6,2017-01-01,113.3
7,2018-01-01,115.2
8,2019-01-01,117.1
9,2020-01-01,118.0


In [91]:
df_CPI['Date'] = pd.to_datetime(df_CPI['Date']).dt.to_period('Y')
df_CPI.set_index('Date').resample('Y')

In [92]:
df_CPI.head(12)

,Date,Sydney_CPI
0,2011,99.8
1,2012,102.3
2,2013,105.0
3,2014,106.8
4,2015,108.9
5,2016,110.9
6,2017,113.3
7,2018,115.2
8,2019,117.1
9,2020,118.0


In [93]:
df_CPI = df_CPI.iloc[:11]

In [94]:
df_CPI= df_CPI.set_index('Date')
df_CPI

,Sydney_CPI
Date,
2011,99.8
2012,102.3
2013,105.0
2014,106.8
2015,108.9
2016,110.9
2017,113.3
2018,115.2
2019,117.1


In [95]:
#Clean GDP data
file_path_3 = Path("real_gdp_2011_2021.csv")
df_gdp = pd.read_csv(file_path_3, index_col='Date', parse_dates=True)

In [96]:
df_gdp.head()

,GDP
Date,
2011-01-01,409213.0
2012-01-01,420816.0
2013-01-01,430860.0
2014-01-01,439841.0
2015-01-01,451668.0


In [97]:
df_gdp = df_gdp.reset_index()

In [98]:
df_gdp['Date'] = pd.to_datetime(df_gdp['Date']).dt.to_period('Y')
df_gdp.set_index('Date').resample('Y')

In [99]:
df_gdp.head()

,Date,GDP
0,2011,409213.0
1,2012,420816.0
2,2013,430860.0
3,2014,439841.0
4,2015,451668.0


In [100]:
df_gdp= df_gdp.set_index('Date')
df_gdp.head(12)

,GDP
Date,
2011,409213.0
2012,420816.0
2013,430860.0
2014,439841.0
2015,451668.0
2016,464007.0
2017,475384.0
2018,486671.0
2019,497078.0


In [101]:
df_gdp = df_gdp.iloc[:11]
df_gdp.head(12)

,GDP
Date,
2011,409213.0
2012,420816.0
2013,430860.0
2014,439841.0
2015,451668.0
2016,464007.0
2017,475384.0
2018,486671.0
2019,497078.0


In [103]:
# macro_df = pd.concat([df_CPI, df_gdp, df_exchange_rates, df_cash_rate],axis=1)
# macro_df.head(11)

In [106]:
pd.concat([df_CPI,df_gdp,df_cash_rate,df_exchange_rates.dropna(),df_dpie],axis=1)
# df_dpie['Detatch'].str.replace(',','')
# df_dpie.dtypes

,Sydney_CPI,GDP,Cash_rate,AUDUSD,Detached
Date,,,,,
2011,99.8,409213.0,4.75,1.0433,526
2012,102.3,420816.0,4.25,1.0393,451
2013,105.0,430860.0,3.00,0.8913,579
2014,106.8,439841.0,2.50,0.8168,919
2015,108.9,451668.0,2.50,0.7275,877
2016,110.9,464007.0,2.00,0.7215,927
2017,113.3,475384.0,1.50,0.7801,816
2018,115.2,486671.0,1.50,0.7049,1060
2019,117.1,497078.0,1.50,0.7021,1094


In [43]:
df_CPI.dtypes

Sydney_CPI    float64
dtype: object

In [44]:
df_cash_rate.dtypes

Cash_rate    float64
dtype: object

In [45]:
df_exchange_rates.dtypes

AUDUSD    float64
dtype: object

In [46]:
df_dpie.dtypes

 Detached     object
dtype: object

In [40]:
# from sklearn.metrics import mean_squared_error

# y_pred = loaded_model.predict(X_test_scaled)

# # Evaluate the model with the MSE metric
# print(mean_squared_error(y_test, y_pred))